<div style="width: 100%; clear: both;">
    <div style="float: left; width: 50%;">
        <img src="../figs/uoc_masterbrand_3linies_positiu.png", align="left">
    </div>
    <div style="float: right; width: 50%;">
        <p style="margin: 0; padding-top: 22px; text-align:right;">M2.855 · Models avançats de mineria de dades</p>
        <p style="margin: 0; text-align:right;">Màster universitari en Ciència de dades (<i>Data science</i>)</p>
        <p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
    </div>
</div>
<div style="width:100%;">&nbsp;</div>

# Exemple de funcionament d'Autoencoder

En aquest exemple veurem un cas d'aplicació per a un autoencoder. En concret, ens centrarem en la compressió i reconstrucció d'imatges fent servir *autoencoders*.

<u>Nota</u>: aquest exemple està basat en https://blog.keras.io/building-autoencoders-in-keras.html

Deshabilitem l'aparició de warnings.

<u>Nota</u>: no es recomana aquest pas quan s'està desenvolupant el codi.

In [ ]:
import warnings

warnings.filterwarnings("ignore")

## Càrrega de les llibreries

Carreguem les llibreries per executar l'exemple.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print("TensorFlow version: {}".format(tf.__version__))
print("Keras version     : {}".format(keras.__version__))
print("Numpy version     : {}".format(np.__version__))

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
import matplotlib.pyplot as plt

## 1. Càrrega del conjunt de dades

En primer lloc, carreguem el conjunt de dades, que en aquest cas serà el dataset de dígits [MNIST](https://en.wikipedia.org/wiki/MNIST_database). 

Realitzem aquesta càrrega, directament, a partir de la llibreria [Keras](https://keras.io/).

In [ ]:
from keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()

In [ ]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

## 2. Creació de l'*autoencoder*

A continuació, creem l'*autoencoder* en la forma més senzilla:
- A partir de les imatges d'entrada de 28x28 píxels a escala de grisos, es crea una capa amb 784 valors d'entrada.
- Creem un **codificador** (*encoder*) amb una representació de 32 neurones.
- Creem un **decodificador** (*decoder*) que generi una sortida a la mateixa resolució que les imatges d'entrada, és a dir, 784 neurones de sortida per generar una imatge de 28x28 píxels en escala de grisos.

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

# Mida de la representació interna de l'autoencoder
layer1dim = 64
layer2dim = 32
layer3dim = 64

# capa d'entrada
input_img = Input(shape=(784,))
# representació interna (encoded)
h1 = Dense(layer1dim, activation="relu")(input_img)
h2 = Dense(layer2dim, activation="relu")(h1)
h3 = Dense(layer3dim, activation="relu")(h2)

# sortida de l'autoencoder, és a dir, la imatge reconstruïda (decoded)
decoded = Dense(784, activation="sigmoid")(h3)

# el model complet d'autoencoder, que apila l'entrada i la sortida
autoencoder = Model(input_img, decoded)

Per a finalitats didàctiques, definim el model `encoder` que genera la codificació interna de l'*autoencoder*, sense el procés de descodificació.

In [ ]:
# aquest model crea la representació interna
encoder = Model(input_img, h3)

encoder.summary()

De manera similar, definim el model `decoder` que, a partir de la representació interna (32 valors), reconstrueix la imatge, és a dir, aplica el procés de descodificació.

In [ ]:
# capa d'entrada de la representació interna
encoded_input = Input(shape=(layer1dim,))

# obtenir la darrera capa de l'autoencoder definit prèviament
decoder_layer = autoencoder.layers[-1]

# creació del model
decoder = Model(encoded_input, decoder_layer(encoded_input))

decoder.summary()

Finalment, definim l'optimitzador i la funció de pèrdua emprada per a l'entrenament de l'*autoencoder*.

In [ ]:
autoencoder.compile(
    loss="binary_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=1.5)
)
autoencoder.summary()

## 3. Entrenament de l'*autoencoder*

En el següent fragment de codi s'entrena l'*autoencoder* creat anteriorment.

Els hiperparàmetres per ajustar l'entrenament, a més de l'optimitzador i la funció de pèrdua, són:
- nombre d'èpoques de l'entrenament
- mida del lot

In [ ]:
n_epochs = 50
n_batch_size = 256

In [ ]:
mfit = autoencoder.fit(
    x_train,
    x_train,
    epochs=n_epochs,
    batch_size=n_batch_size,
    shuffle=True,
    validation_data=(x_test, x_test),
)

In [ ]:
# Stacked Autoencoder with functional model
# encoder
inputs = keras.Input(shape=(28, 28))
lr_flatten = keras.layers.Flatten()(inputs)
lr1 = keras.layers.Dense(392, activation="selu")(lr_flatten)
lr2 = keras.layers.Dense(196, activation="selu")(lr1)
# decoder
lr3 = keras.layers.Dense(392, activation="selu")(lr2)
lr4 = keras.layers.Dense(28 * 28, activation="sigmoid")(lr3)
outputs = keras.layers.Reshape([28, 28])(lr4)
stacked_ae = keras.models.Model(inputs, outputs)
stacked_ae.compile(
    loss="binary_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=1.5)
)
stacked_ae.summary()

In [ ]:
import matplotlib.pyplot as plt

# Plot del training loss
plt.style.use("ggplot")

plt.plot(np.arange(0, n_epochs), mfit.history["loss"], label="train")
plt.plot(np.arange(0, n_epochs), mfit.history["val_loss"], label="val")
plt.title("Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.show()

## 4. Visualització dels resultats

Finalment, presentarem els resultats d'aplicar l'autoencoder en alguns exemples del conjunt de dades de test.

In [ ]:
# codifiquem i descodifiquem les imatges de test
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

Noteu que aquest pas es pot fer en un sol moviment fent servir:

> decoded_imgs = autoencoder.predict(x_test)

In [ ]:
import matplotlib.pyplot as plt

n = 10  # quantitat d'exemples per mostrar
plt.figure(figsize=(20, 4))

for i in range(n):
    # mostrar la imatge original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # mostrar la reconstrucció
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()